In [ ]:
# @title 1. Initialize & Hardware Check
import os
import torch
from google.colab import drive

# 1. Mount Drive
drive.mount('/content/drive')

# 2. Clone/Update Repo
REPO_URL = "https://github.com/BorisKlimchenko/Adaptive-Motion-Lab.git"
REPO_DIR = "/content/Adaptive-Motion-Lab"

if not os.path.exists(REPO_DIR):
    print(f"Cloning {REPO_URL}...")
    !git clone {REPO_URL}
else:
    print("Pulling latest changes...")
    !cd {REPO_DIR} && git pull

# 3. Hardware Check
print("\n--- Hardware Profile ---")
if torch.cuda.is_available():
    gpu = torch.cuda.get_device_name(0)
    print(f"GPU: {gpu}")
    if "T4" in gpu: print("ℹ️ Mode: Efficiency (T4)")
    elif "A100" in gpu: print("🚀 Mode: Performance (A100)")
else:
    print("⚠️ No GPU detected!")

%cd {REPO_DIR}

In [ ]:
# @title ⚡ Установка окружения (The Golden Trio)
# @markdown Этот скрипт удаляет конфликтующие версии и ставит строго:
# @markdown - PyTorch 2.5.1 (CUDA 12.1)
# @markdown - xFormers 0.0.28
# @markdown - Diffusers & Transformers

import subprocess
import sys

def run_command(command, msg):
    print(f"🔄 {msg}...")
    try:
        subprocess.check_call(command, shell=True)
        print(f"✅ Успешно.")
    except subprocess.CalledProcessError:
        print(f"❌ Ошибка при выполнении: {msg}")
        sys.exit(1) # Останавливаем выполнение, если база не встала

# 1. ЗАЧИСТКА (Nuke & Pave)
# Удаляем всё, что может конфликтовать. Это критически важно для Colab.
run_command(
    "pip uninstall -y torch torchvision torchaudio xformers",
    "Очистка старого окружения PyTorch"
)

# 2. ФУНДАМЕНТ (Golden Trio)
# Ставим конкретные версии, которые гарантированно работают вместе.
# --no-deps для xformers важно, чтобы он не тянул свои зависимости.
run_command(
    "pip install torch==2.5.1+cu121 torchvision==0.20.1+cu121 torchaudio==2.5.1+cu121 xformers==0.0.28.post3 --index-url https://download.pytorch.org/whl/cu121",
    "Установка ядра: Torch 2.5.1 + xFormers 0.0.28"
)

# 3. ПРИЛОЖЕНИЕ
# Теперь накатываем остальные библиотеки из requirements.txt
run_command(
    "pip install -r requirements.txt",
    "Установка библиотек проекта (Diffusers, etc.)"
)

In [ ]:
# @title 3. Run Inference Engine
# Здесь мы используем новый путь к конфигу, который мы только что структурировали!
CONFIG_PATH = "configs/default_scene.json" 

print(f"🚀 Launching Engine with: {CONFIG_PATH}")
!python main.py --prompts {CONFIG_PATH}